In [1]:
import string
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from nltk.corpus import stopwords
from nltk.util import ngrams
from sklearn.feature_extraction.text import CountVectorizer
from collections import defaultdict
from collections import  Counter
plt.style.use('ggplot')
stop=set(stopwords.words('english'))
import re
from nltk.tokenize import word_tokenize
import gensim
import string
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
import pickle
from keras.models import load_model
from keras.initializers import Constant
from BetterRnnlm import BetterRnnlm
from sklearn.model_selection import train_test_split
from SGD import SGD
from RnnlmTrainer import RnnlmTrainer

In [2]:
tweet = pd.read_csv('../dataset/disaster/nlp_getting_started/train.csv')
test=pd.read_csv('../dataset/disaster/nlp_getting_started/test.csv')
tweet.head(3)

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1


In [3]:
df=pd.concat([tweet,test])
df.shape

(10876, 5)

In [4]:
def remove_URL(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'',text)

In [5]:
df['text']=df['text'].apply(lambda x : remove_URL(x))

In [6]:
def remove_html(text):
    html=re.compile(r'<.*?>')
    return html.sub(r'',text)

In [7]:
df['text']=df['text'].apply(lambda x : remove_html(x))

In [8]:
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

In [9]:
df['text']=df['text'].apply(lambda x: remove_emoji(x))

In [10]:
def remove_punct(text):
    table=str.maketrans('','',string.punctuation)
    return text.translate(table)

In [11]:
df['text']=df['text'].apply(lambda x : remove_punct(x))

In [12]:
def create_corpus(df):
    corpus=[]
    for tweet in df['text']:
        words=[word.lower() for word in word_tokenize(tweet) if((word.isalpha()==1) & (word not in stop))]
        corpus.append(words)
    return corpus

In [13]:
corpus = create_corpus(df)

In [14]:
corpus[:5]

[['our', 'deeds', 'reason', 'earthquake', 'may', 'allah', 'forgive', 'us'],
 ['forest', 'fire', 'near', 'la', 'ronge', 'sask', 'canada'],
 ['all',
  'residents',
  'asked',
  'shelter',
  'place',
  'notified',
  'officers',
  'no',
  'evacuation',
  'shelter',
  'place',
  'orders',
  'expected'],
 ['people', 'receive', 'wildfires', 'evacuation', 'orders', 'california'],
 ['just',
  'got',
  'sent',
  'photo',
  'ruby',
  'alaska',
  'smoke',
  'wildfires',
  'pours',
  'school']]

In [15]:
with open('../Disaster_Tweet/model/tokenize_model/tokenizer_obj.pkl', 'rb') as file:
    tokenizer_obj = pickle.load(file)

In [17]:
word_index = tokenizer_obj.word_index
batch_size = 20
wordvec_size = 50
hidden_size = 50
time_size = 50
lr = 20.0
max_epoch = 40
max_grad = 0.25
dropout = 0.5
vocab_size = len(word_index)


In [18]:
tokenizer_obj.fit_on_texts(corpus)
sequences = tokenizer_obj.texts_to_sequences(corpus)
tweet_pad = pad_sequences(sequences, maxlen = time_size, truncating = 'post', padding = 'post')

In [ ]:
model = BetterRnnlm(vocab_size, wordvec_size, hidden_size, dropout)

In [ ]:
optimizer = SGD(lr)

In [ ]:
trainer = RnnlmTrainer(model, optimizer)

In [ ]:
train=tweet_pad[:tweet.shape[0]]
text=tweet_pad[tweet.shape[0]:]

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(train,tweet['target'].values,test_size=0.15)
print('Shape of train',X_train.shape)
print("Shape of Validation ",X_test.shape)

In [ ]:
best_ppl = float('inf')

In [ ]:
for epoch in range(max_epoch):
    trainer.fit(X_train, y_train, max_epoch = 1, batch_size = batch_size,
               time_size = time_size, max_grad = max_grad)
    model.reset_state()
    ppl = eval_perplexity(model, )